# Telecom Churn case study using PCA and Ensemble

### Problem Statement
In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business
goal. To reduce customer churn, telecom companies need to predict which customers are at high risk of churn. In this project, you will analyze customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn.

In this competition, your goal is to build a machine learning model that is able to predict churning customers based on the features provided for their usage.

### Goal
It is your job to predict if a customer will churn, given the ~170 columns containing customer behavior, usage patterns, payment patterns, and other features that might be relevant. Your target variable is "churn_probability"
Note: Make sure your accuracy is greater than the sample submission that is present in the leaderboard

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

## Step 1. Reading, Understanding and Visualizing the data

In [2]:
telecom_train_df = pd.read_csv('train (1).csv')
telecom_test_df = pd.read_csv('test (1).csv')

In [3]:
telecom_train_df.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,...,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,...,0,0,NaN,1.0,NaN,710,0.0,0.0,0.0,0
2,2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,...,0,0,NaN,NaN,NaN,882,0.0,0.0,0.0,0
3,3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,...,0,0,NaN,NaN,NaN,982,0.0,0.0,0.0,0
4,4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,...,1,0,1.0,1.0,1.0,647,0.0,0.0,0.0,0


In [4]:
telecom_test_df.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g
0,69999,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,91.882,65.330,...,0,0,0,NaN,NaN,NaN,1692,0.00,0.00,0.00
1,70000,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,414.168,515.568,...,0,0,0,NaN,NaN,NaN,2533,0.00,0.00,0.00
2,70001,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,329.844,434.884,...,0,0,0,NaN,NaN,NaN,277,525.61,758.41,241.84
3,70002,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,43.550,171.390,...,0,0,0,NaN,NaN,NaN,1244,0.00,0.00,0.00
4,70003,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,306.854,406.289,...,0,0,0,NaN,NaN,NaN,462,0.00,0.00,0.00


In [5]:
telecom_train_df.shape

(69999, 172)

In [6]:
telecom_test_df.shape

(30000, 171)

In [7]:
telecom_train_df.columns

Index(['id', 'circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou',
       'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8',
       'arpu_6', 'arpu_7',
       ...
       'sachet_3g_7', 'sachet_3g_8', 'fb_user_6', 'fb_user_7', 'fb_user_8',
       'aon', 'aug_vbc_3g', 'jul_vbc_3g', 'jun_vbc_3g', 'churn_probability'],
      dtype='object', length=172)

In [8]:
telecom_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Columns: 172 entries, id to churn_probability
dtypes: float64(135), int64(28), object(9)
memory usage: 91.9+ MB


In [9]:
telecom_train_df.describe()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
count,69999.000000,69999.0,69297.0,69297.0,69297.0,69999.000000,69999.000000,69999.000000,67231.000000,67312.000000,...,69999.000000,69999.000000,17568.000000,17865.000000,18417.000000,69999.000000,69999.000000,69999.000000,69999.00000,69999.000000
mean,34999.000000,109.0,0.0,0.0,0.0,283.134365,278.185912,278.858826,133.153275,133.894438,...,0.081444,0.085487,0.916325,0.909544,0.890319,1220.639709,68.108597,65.935830,60.07674,0.101887
std,20207.115084,0.0,0.0,0.0,0.0,334.213918,344.366927,351.924315,299.963093,311.277193,...,0.634547,0.680035,0.276907,0.286842,0.312501,952.426321,269.328659,267.899034,257.22681,0.302502
min,0.000000,109.0,0.0,0.0,0.0,-2258.709000,-1289.715000,-945.808000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.00000,0.000000
25%,17499.500000,109.0,0.0,0.0,0.0,93.581000,86.714000,84.095000,7.410000,6.675000,...,0.000000,0.000000,1.000000,1.000000,1.000000,468.000000,0.000000,0.000000,0.00000,0.000000
50%,34999.000000,109.0,0.0,0.0,0.0,197.484000,191.588000,192.234000,34.110000,32.280000,...,0.000000,0.000000,1.000000,1.000000,1.000000,868.000000,0.000000,0.000000,0.00000,0.000000
75%,52498.500000,109.0,0.0,0.0,0.0,370.791000,365.369500,369.909000,119.390000,115.837500,...,0.000000,0.000000,1.000000,1.000000,1.000000,1813.000000,0.000000,0.000000,0.00000,0.000000
max,69998.000000,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,7376.710000,8157.780000,...,33.000000,41.000000,1.000000,1.000000,1.000000,4337.000000,12916.220000,9165.600000,11166.21000,1.000000


In [10]:
telecom_train_df.isna().sum()

id                     0
circle_id              0
loc_og_t2o_mou       702
std_og_t2o_mou       702
loc_ic_t2o_mou       702
                    ... 
aon                    0
aug_vbc_3g             0
jul_vbc_3g             0
jun_vbc_3g             0
churn_probability      0
Length: 172, dtype: int64

In [11]:
telecom_train_df.dtypes

id                     int64
circle_id              int64
loc_og_t2o_mou       float64
std_og_t2o_mou       float64
loc_ic_t2o_mou       float64
                      ...   
aon                    int64
aug_vbc_3g           float64
jul_vbc_3g           float64
jun_vbc_3g           float64
churn_probability      int64
Length: 172, dtype: object

### Dropping rows for both train and test datasets
 - Rows which are empty

In [12]:
telecom_train_df.dropna(axis=0, how='all', inplace=True)
telecom_test_df.dropna(axis=0, how='all', inplace=True)

In [13]:
print(telecom_train_df.shape)
print(telecom_test_df.shape)

(69999, 172)
(30000, 171)


### Dropping columns both train and test datasets

    - Columns having more than 70% values as null
    - Not needed like Id

In [14]:
# to check the % of null values
round((pd.isnull(telecom_train_df).sum()/len(telecom_train_df.index)),2)[pd.isnull(telecom_train_df).sum()/ len(telecom_train_df.index) > 0.60].sort_values()

count_rech_3g_8             0.74
night_pck_user_8            0.74
night_pck_user_7            0.74
arpu_2g_8                   0.74
arpu_2g_7                   0.74
arpu_3g_8                   0.74
arpu_3g_7                   0.74
av_rech_amt_data_8          0.74
av_rech_amt_data_7          0.74
fb_user_7                   0.74
count_rech_3g_7             0.74
fb_user_8                   0.74
total_rech_data_8           0.74
count_rech_2g_7             0.74
date_of_last_rech_data_7    0.74
max_rech_data_8             0.74
max_rech_data_7             0.74
date_of_last_rech_data_8    0.74
count_rech_2g_8             0.74
total_rech_data_7           0.74
fb_user_6                   0.75
night_pck_user_6            0.75
total_rech_data_6           0.75
count_rech_3g_6             0.75
max_rech_data_6             0.75
arpu_3g_6                   0.75
av_rech_amt_data_6          0.75
count_rech_2g_6             0.75
arpu_2g_6                   0.75
date_of_last_rech_data_6    0.75
dtype: flo

In [15]:
len(round((pd.isnull(telecom_train_df).sum()/len(telecom_train_df.index)),2)[pd.isnull(telecom_train_df).sum()/ len(telecom_train_df.index) > 0.70].sort_values())

30

In [16]:
len(round((pd.isnull(telecom_test_df).sum()/len(telecom_test_df.index)),2)[pd.isnull(telecom_test_df).sum()/ len(telecom_test_df.index) > 0.70].sort_values())

30

### As we have more than 70% data as null for 30 columns in train and test, we are dropping these columns from dataset.

In [17]:
cols_to_delete = ((pd.isnull(telecom_train_df).sum()/len(telecom_train_df.index))[pd.isnull(telecom_train_df).sum()/ len(telecom_train_df.index) > 0.60].sort_values()).keys()

In [18]:
cols_to_delete = cols_to_delete.to_list()

In [19]:
telecom_train_df = telecom_train_df.drop(cols_to_delete, axis=1)

In [20]:
telecom_train_df.shape

(69999, 142)

In [21]:
telecom_test_df = telecom_test_df.drop(cols_to_delete, axis=1)

In [22]:
telecom_test_df.shape

(30000, 141)

## Splitting the data in numerical and categorical for train & test dataset

In [23]:
train_datatype_columns = telecom_train_df.dtypes #type of each feature in data: int, float, object
train_numerical_columns = train_datatype_columns[(train_datatype_columns == 'int64') | (train_datatype_columns == 'float64')]
train_numerical_columns = train_numerical_columns.index.to_list()
train_categorical_columns = train_datatype_columns[train_datatype_columns == 'object']
train_categorical_columns = train_categorical_columns.index.to_list()

In [24]:
test_datatype_columns = telecom_test_df.dtypes #type of each feature in data: int, float, object
test_numerical_columns = test_datatype_columns[(test_datatype_columns == 'int64') | (test_datatype_columns == 'float64')] 
test_numerical_columns = test_numerical_columns.index.to_list()
test_categorical_columns = test_datatype_columns[test_datatype_columns == 'object']
test_categorical_columns = test_categorical_columns.index.to_list()

### Imputing the na values with
 - Median for numerical variables
 - Mode for categorical variables

In [25]:
# A generic method to impute na
def replace_na(df, columns, data_type):
    for col in columns:
        if data_type == 'categorical':
            value = pd.to_datetime(df[col]).mode()
        else:
            print(col)
            value = df[col].median()
        df[col].fillna(value, inplace=True)
    return df

In [26]:
telecom_train_df = replace_na(telecom_train_df, train_categorical_columns, 'categorical')
telecom_train_df = replace_na(telecom_train_df, train_numerical_columns, 'numerical')

id
circle_id
loc_og_t2o_mou
std_og_t2o_mou
loc_ic_t2o_mou
arpu_6
arpu_7
arpu_8
onnet_mou_6
onnet_mou_7
onnet_mou_8
offnet_mou_6
offnet_mou_7
offnet_mou_8
roam_ic_mou_6
roam_ic_mou_7
roam_ic_mou_8
roam_og_mou_6
roam_og_mou_7
roam_og_mou_8
loc_og_t2t_mou_6
loc_og_t2t_mou_7
loc_og_t2t_mou_8
loc_og_t2m_mou_6
loc_og_t2m_mou_7
loc_og_t2m_mou_8
loc_og_t2f_mou_6
loc_og_t2f_mou_7
loc_og_t2f_mou_8
loc_og_t2c_mou_6
loc_og_t2c_mou_7
loc_og_t2c_mou_8
loc_og_mou_6
loc_og_mou_7
loc_og_mou_8
std_og_t2t_mou_6
std_og_t2t_mou_7
std_og_t2t_mou_8
std_og_t2m_mou_6
std_og_t2m_mou_7
std_og_t2m_mou_8
std_og_t2f_mou_6
std_og_t2f_mou_7
std_og_t2f_mou_8
std_og_t2c_mou_6
std_og_t2c_mou_7
std_og_t2c_mou_8
std_og_mou_6
std_og_mou_7
std_og_mou_8
isd_og_mou_6
isd_og_mou_7
isd_og_mou_8
spl_og_mou_6
spl_og_mou_7
spl_og_mou_8
og_others_6
og_others_7
og_others_8
total_og_mou_6
total_og_mou_7
total_og_mou_8
loc_ic_t2t_mou_6
loc_ic_t2t_mou_7
loc_ic_t2t_mou_8
loc_ic_t2m_mou_6
loc_ic_t2m_mou_7
loc_ic_t2m_mou_8
loc_ic_t2f_mou_

In [30]:
# Lets impute the values for test data as well
telecom_test_df = replace_na(telecom_test_df, test_categorical_columns, 'categorical')
telecom_test_df = replace_na(telecom_test_df, test_numerical_columns, 'numerical')


id
circle_id
loc_og_t2o_mou
std_og_t2o_mou
loc_ic_t2o_mou
arpu_6
arpu_7
arpu_8
onnet_mou_6
onnet_mou_7
onnet_mou_8
offnet_mou_6
offnet_mou_7
offnet_mou_8
roam_ic_mou_6
roam_ic_mou_7
roam_ic_mou_8
roam_og_mou_6
roam_og_mou_7
roam_og_mou_8
loc_og_t2t_mou_6
loc_og_t2t_mou_7
loc_og_t2t_mou_8
loc_og_t2m_mou_6
loc_og_t2m_mou_7
loc_og_t2m_mou_8
loc_og_t2f_mou_6
loc_og_t2f_mou_7
loc_og_t2f_mou_8
loc_og_t2c_mou_6
loc_og_t2c_mou_7
loc_og_t2c_mou_8
loc_og_mou_6
loc_og_mou_7
loc_og_mou_8
std_og_t2t_mou_6
std_og_t2t_mou_7
std_og_t2t_mou_8
std_og_t2m_mou_6
std_og_t2m_mou_7
std_og_t2m_mou_8
std_og_t2f_mou_6
std_og_t2f_mou_7
std_og_t2f_mou_8
std_og_t2c_mou_6
std_og_t2c_mou_7
std_og_t2c_mou_8
std_og_mou_6
std_og_mou_7
std_og_mou_8
isd_og_mou_6
isd_og_mou_7
isd_og_mou_8
spl_og_mou_6
spl_og_mou_7
spl_og_mou_8
og_others_6
og_others_7
og_others_8
total_og_mou_6
total_og_mou_7
total_og_mou_8
loc_ic_t2t_mou_6
loc_ic_t2t_mou_7
loc_ic_t2t_mou_8
loc_ic_t2m_mou_6
loc_ic_t2m_mou_7
loc_ic_t2m_mou_8
loc_ic_t2f_mou_

### Lets check for unique entries in train and test dataset

In [28]:
# For train data
for column in train_numerical_columns:
    print (f"{column} has number of categories as : {telecom_train_df[column].unique()}")
    print (f"{telecom_train_df[column].value_counts()}\n")

id has number of categories as : [    0     1     2 ... 69996 69997 69998]
0        1
46664    1
46670    1
46669    1
46668    1
        ..
23338    1
23339    1
23340    1
23341    1
69998    1
Name: id, Length: 69999, dtype: int64

circle_id has number of categories as : [109]
109    69999
Name: circle_id, dtype: int64

loc_og_t2o_mou has number of categories as : [0.]
0.0    69999
Name: loc_og_t2o_mou, dtype: int64

std_og_t2o_mou has number of categories as : [0.]
0.0    69999
Name: std_og_t2o_mou, dtype: int64

loc_ic_t2o_mou has number of categories as : [0.]
0.0    69999
Name: loc_ic_t2o_mou, dtype: int64

arpu_6 has number of categories as : [ 31.277   0.     60.806 ... 372.088 238.575 168.269]
 0.000      2603
 20.000       72
-5.000        72
 1.000        49
 1.500        38
            ... 
 119.580       1
 393.027       1
 48.879        1
 179.191       1
 168.269       1
Name: arpu_6, Length: 61615, dtype: int64

arpu_7 has number of categories as : [ 87.009 122.787 103

0.00      5700
63.52     3703
0.01        29
0.23        28
0.43        27
          ... 
407.68       1
541.99       1
256.56       1
149.18       1
342.29       1
Name: loc_og_mou_8, Length: 21951, dtype: int64

std_og_t2t_mou_6 has number of categories as : [4.6340e+01 0.0000e+00 2.6000e-01 ... 3.3709e+02 7.4931e+02 2.5811e+02]
0.00      39256
0.48         52
0.43         49
0.30         43
0.01         43
          ...  
539.78        1
377.26        1
96.83         1
93.63         1
258.11        1
Name: std_og_t2t_mou_6, Length: 14554, dtype: int64

std_og_t2t_mou_7 has number of categories as : [124.38   0.     2.94 ... 461.18 780.59 427.58]
0.00      39364
0.66         47
0.36         46
1.01         45
0.26         43
          ...  
715.14        1
16.11         1
586.68        1
570.03        1
427.58        1
Name: std_og_t2t_mou_7, Length: 14745, dtype: int64

std_og_t2t_mou_8 has number of categories as : [  1.01   0.    95.03 ... 132.16 306.16 428.68]
0.00      40215
0.3

0.00      40858
0.03        236
0.01        153
0.05        133
0.23        121
          ...  
66.81         1
44.08         1
53.23         1
48.31         1
283.79        1
Name: std_ic_t2t_mou_7, Length: 5479, dtype: int64

std_ic_t2t_mou_8 has number of categories as : [  0.21   0.     2.95 ...  33.53 164.66  65.14]
0.00      41271
0.03        207
0.01        148
0.05        141
0.06        129
          ...  
157.08        1
124.08        1
162.31        1
59.76         1
65.14         1
Name: std_ic_t2t_mou_8, Length: 5367, dtype: int64

std_ic_t2m_mou_6 has number of categories as : [  7.46   0.    18.21 ... 148.74  34.34 144.89]
0.00      24167
2.04       2791
0.03        150
0.31        138
0.35        135
          ...  
311.14        1
178.24        1
53.39         1
192.29        1
144.89        1
Name: std_ic_t2m_mou_6, Length: 7931, dtype: int64

std_ic_t2m_mou_7 has number of categories as : [ 19.96   0.     2.48 ...  97.13 406.19 166.54]
0.00      24315
2.06       2732

0.00      58658
0.01        111
0.46         71
0.05         69
0.03         66
          ...  
62.68         1
161.01        1
120.43        1
22.51         1
562.11        1
Name: isd_ic_mou_7, Length: 4640, dtype: int64

isd_ic_mou_8 has number of categories as : [  0.   402.38   1.1  ... 947.44  67.71  35.79]
0.00      59051
0.46         95
0.01         83
0.03         74
0.05         66
          ...  
85.86         1
78.73         1
21.64         1
177.28        1
35.79         1
Name: isd_ic_mou_8, Length: 4694, dtype: int64

ic_others_6 has number of categories as : [ 1.11  0.    0.18 ... 56.19 42.01 17.46]
0.00     54453
0.06       551
0.48       527
0.05       500
0.15       475
         ...  
42.63        1
11.15        1
59.09        1
17.19        1
17.46        1
Name: ic_others_6, Length: 1523, dtype: int64

ic_others_7 has number of categories as : [6.9000e-01 0.0000e+00 1.9000e-01 ... 1.5090e+01 1.0350e+01 6.1349e+02]
0.00      56437
0.05        297
0.06        284
0.1

## Lets focus on Outliers now and treat them

In [29]:
# Generic method to remove outliers
def remove_outliers(df, features):
    for feature in features:
        q1 = df[feature].quantile(0.25)
        q3 = df[feature].quantile(0.99)
        iqr = q3-q1
        lower_value  = q1 - (1.5 * iqr)
        higer_value = q3 + (1.5 * iqr)
        df = df[(df[feature] <= higer_value) & (df[feature] >= lower_value)]
    return df